In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
BATCH_SIZE = 32  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels
total_train = 10343
total_val = 4424

#DESIRED_ACCURACY = 0.98

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        #if(logs.get('accuracy') > DESIRED_ACCURACY):
        if logs.get('acc') is not None and logs.get('acc') > 0.98:
            print("\nReached 98% accuracy so cencelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [6]:
train_image_generator = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_image_generator = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [14]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory='/content/drive/My Drive/CSE498_Rashed_Samina_Hafiz_Tushar/Natural_Face_CNN/train',
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), 
                                                           shuffle=True,
                                                           class_mode='binary')

Found 10343 images belonging to 7 classes.


In [15]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory='/content/drive/My Drive/CSE498_Rashed_Samina_Hafiz_Tushar/Natural_Face_CNN/test',
                                                              shuffle=False,
                                                              target_size=(IMG_SHAPE,IMG_SHAPE), 
                                                              class_mode='binary')

Found 4424 images belonging to 7 classes.


In [7]:
callbacks = myCallback()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation='sigmoid')
])
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [8]:
from keras import backend as K

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [9]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy',precision_m])

In [14]:
epochs = 10
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
324/324 [==============================] - 3557s 11s/step - loss: 1.9505 - accuracy: 0.2273 - precision_m: 1.3587 - val_loss: 1.8463 - val_accuracy: 0.2636 - val_precision_m: 1.5202
Epoch 2/10
324/324 [==============================] - 317s 977ms/step - loss: 1.8287 - accuracy: 0.2796 - precision_m: 1.4496 - val_loss: 1.7847 - val_accuracy: 0.2862 - val_precision_m: 1.5115
Epoch 3/10
324/324 [==============================] - 315s 973ms/step - loss: 1.7786 - accuracy: 0.2968 - precision_m: 1.6274 - val_loss: 1.7478 - val_accuracy: 0.3162 - val_precision_m: 1.6606
Epoch 4/10
324/324 [==============================] - 314s 968ms/step - loss: 1.7251 - accuracy: 0.3153 - precision_m: 1.6420 - val_loss: 1.6547 - val_accuracy: 0.3562 - val_precision_m: 1.6450
Epoch 5/10
324/324 [==============================] - 311s 961ms/step - loss: 1.6640 - accuracy: 0.3491 - precision_m: 1.6284 - val_loss: 1.6530 - val_accuracy: 0.3526 - val_precision_m: 1.6558
Epoch 6/10
324/324 [===========

# Until 10th epochs : accuracy is 41.94%
# 317s 978ms/step - loss: 1.4832 - accuracy: 0.4194 - precision_m: 1.5418 - val_loss: 1.4968 - val_accuracy: 0.4170 - val_precision_m: 1.6642

In [15]:
epochs = 15
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
324/324 [==============================] - 324s 1s/step - loss: 1.4711 - accuracy: 0.4388 - precision_m: 1.5467 - val_loss: 1.4708 - val_accuracy: 0.4315 - val_precision_m: 1.6563
Epoch 2/15
324/324 [==============================] - 323s 996ms/step - loss: 1.4454 - accuracy: 0.4394 - precision_m: 1.5529 - val_loss: 1.4257 - val_accuracy: 0.4433 - val_precision_m: 1.5052
Epoch 3/15
324/324 [==============================] - 325s 1s/step - loss: 1.4315 - accuracy: 0.4511 - precision_m: 1.5293 - val_loss: 1.4241 - val_accuracy: 0.4464 - val_precision_m: 1.5659
Epoch 4/15
324/324 [==============================] - 323s 996ms/step - loss: 1.4074 - accuracy: 0.4584 - precision_m: 1.5013 - val_loss: 1.4136 - val_accuracy: 0.4516 - val_precision_m: 1.5055
Epoch 5/15
324/324 [==============================] - 323s 996ms/step - loss: 1.3981 - accuracy: 0.4635 - precision_m: 1.5053 - val_loss: 1.4044 - val_accuracy: 0.4539 - val_precision_m: 1.5614
Epoch 6/15
324/324 [================

# Until 25th epochs : accuracy is 51.63%
# 321s 989ms/step - loss: 1.2716 - accuracy: 0.5163 - precision_m: 1.4859 - val_loss: 1.3431 - val_accuracy: 0.4817 - val_precision_m: 1.5326

In [18]:
epochs = 20
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
324/324 [==============================] - 322s 992ms/step - loss: 1.1620 - accuracy: 0.5624 - precision_m: 1.4990 - val_loss: 1.2199 - val_accuracy: 0.5398 - val_precision_m: 1.5388
Epoch 2/20
324/324 [==============================] - 323s 996ms/step - loss: 1.1486 - accuracy: 0.5684 - precision_m: 1.4832 - val_loss: 1.2114 - val_accuracy: 0.5427 - val_precision_m: 1.5276
Epoch 3/20
324/324 [==============================] - 324s 999ms/step - loss: 1.1474 - accuracy: 0.5706 - precision_m: 1.4889 - val_loss: 1.2364 - val_accuracy: 0.5384 - val_precision_m: 1.5608
Epoch 4/20
324/324 [==============================] - 324s 1s/step - loss: 1.1377 - accuracy: 0.5757 - precision_m: 1.4844 - val_loss: 1.2046 - val_accuracy: 0.5500 - val_precision_m: 1.5599
Epoch 5/20
324/324 [==============================] - 328s 1s/step - loss: 1.1490 - accuracy: 0.5671 - precision_m: 1.4963 - val_loss: 1.2183 - val_accuracy: 0.5429 - val_precision_m: 1.5090
Epoch 6/20
324/324 [================

# Until 45th epochs : accuracy is 60.70%
# 321s 989ms/step - loss: 1.0692 - accuracy: 0.6070 - precision_m: 1.4770 - val_loss: 1.1639 - val_accuracy: 0.5615 - val_precision_m: 1.4497

In [19]:
epochs = 15
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
324/324 [==============================] - 324s 1s/step - loss: 1.0772 - accuracy: 0.5963 - precision_m: 1.4612 - val_loss: 1.1469 - val_accuracy: 0.5649 - val_precision_m: 1.5346
Epoch 2/15
324/324 [==============================] - 321s 989ms/step - loss: 1.0643 - accuracy: 0.6008 - precision_m: 1.4761 - val_loss: 1.1409 - val_accuracy: 0.5717 - val_precision_m: 1.4327
Epoch 3/15
324/324 [==============================] - 319s 985ms/step - loss: 1.0619 - accuracy: 0.6043 - precision_m: 1.4722 - val_loss: 1.1497 - val_accuracy: 0.5633 - val_precision_m: 1.4947
Epoch 4/15
324/324 [==============================] - 319s 984ms/step - loss: 1.0658 - accuracy: 0.6013 - precision_m: 1.4816 - val_loss: 1.1401 - val_accuracy: 0.5737 - val_precision_m: 1.5481
Epoch 5/15
324/324 [==============================] - 320s 987ms/step - loss: 1.0564 - accuracy: 0.6108 - precision_m: 1.4674 - val_loss: 1.1522 - val_accuracy: 0.5698 - val_precision_m: 1.4617
Epoch 6/15
324/324 [=============

# Until 60th epochs : accuracy is 61.39%
# 323s 997ms/step - loss: 1.0289 - accuracy: 0.6139 - precision_m: 1.4534 - val_loss: 1.1818 - val_accuracy: 0.5642 - val_precision_m: 1.4233

In [20]:
epochs = 5
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
324/324 [==============================] - 327s 1s/step - loss: 1.0239 - accuracy: 0.6205 - precision_m: 1.4606 - val_loss: 1.1340 - val_accuracy: 0.5848 - val_precision_m: 1.4831
Epoch 2/5
324/324 [==============================] - 325s 1s/step - loss: 1.0346 - accuracy: 0.6124 - precision_m: 1.4625 - val_loss: 1.1442 - val_accuracy: 0.5753 - val_precision_m: 1.4643
Epoch 3/5
324/324 [==============================] - 328s 1s/step - loss: 1.0053 - accuracy: 0.6269 - precision_m: 1.4588 - val_loss: 1.1428 - val_accuracy: 0.5759 - val_precision_m: 1.4956
Epoch 4/5
324/324 [==============================] - 327s 1s/step - loss: 1.0154 - accuracy: 0.6239 - precision_m: 1.4571 - val_loss: 1.1341 - val_accuracy: 0.5863 - val_precision_m: 1.4905
Epoch 5/5
324/324 [==============================] - 325s 1s/step - loss: 1.0096 - accuracy: 0.6252 - precision_m: 1.4563 - val_loss: 1.1382 - val_accuracy: 0.5766 - val_precision_m: 1.5466


# Until 65th epochs : accuracy is 62.52%
# 325s 1s/step - loss: 1.0096 - accuracy: 0.6252 - precision_m: 1.4563 - val_loss: 1.1382 - val_accuracy: 0.5766 - val_precision_m: 1.5466

In [21]:

epochs = 15
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
324/324 [==============================] - 327s 1s/step - loss: 1.0057 - accuracy: 0.6228 - precision_m: 1.4694 - val_loss: 1.2109 - val_accuracy: 0.5676 - val_precision_m: 1.4163
Epoch 2/15
324/324 [==============================] - 324s 999ms/step - loss: 1.0131 - accuracy: 0.6224 - precision_m: 1.4501 - val_loss: 1.1764 - val_accuracy: 0.5694 - val_precision_m: 1.4761
Epoch 3/15
324/324 [==============================] - 325s 1s/step - loss: 1.0160 - accuracy: 0.6245 - precision_m: 1.4653 - val_loss: 1.0982 - val_accuracy: 0.6051 - val_precision_m: 1.4937
Epoch 4/15
324/324 [==============================] - 325s 1s/step - loss: 0.9951 - accuracy: 0.6312 - precision_m: 1.4537 - val_loss: 1.1306 - val_accuracy: 0.5891 - val_precision_m: 1.4604
Epoch 5/15
324/324 [==============================] - 321s 991ms/step - loss: 1.0137 - accuracy: 0.6252 - precision_m: 1.4531 - val_loss: 1.1602 - val_accuracy: 0.5739 - val_precision_m: 1.4287
Epoch 6/15
324/324 [===================

# Until 80th epochs : accuracy is 63.81%
# 321s 992ms/step - loss: 0.9738 - accuracy: 0.6381 - precision_m: 1.4578 - val_loss: 1.0949 - val_accuracy: 0.6065 - val_precision_m: 1.4654